In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sqlite3
import sys
import logging
import json

# Data
from sklearn.model_selection import train_test_split

# Tensorflow
import tensorflow as tf

# Graph
import tensorflow_gnn as tfgnn

# Add the project root to the Python path
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from scripts.data_cleaner import filter_top_cpv_categories
from scripts.gnn_anomaly_detection import ProcurementGraphBuilder, GNNAnomalyDetector, AnomalyAnalyzer

from scripts.preprocess_pipeline import create_pipeline_cat


%load_ext autoreload
%autoreload 2


2025-06-06 16:22:06.406122: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-06 16:22:06.417343: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-06 16:22:06.493239: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-06 16:22:06.493305: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-06 16:22:06.493435: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
graph_builder = ProcurementGraphBuilder()

data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')

df = graph_builder.load_data(data_path)

X_train_preproc, X_test_preproc, X_train, X_test = graph_builder.preprocess_data(df)



INFO:scripts.gnn_anomaly_detection:Loading data from /home/ronan/code/RonanB400/Project/decp_ml/data
INFO:scripts.gnn_anomaly_detection:Preprocessing data...


Filtered from 392 to 60 CPV categories, keeping 250895 rows out of 286850


/home/ronan/.pyenv/versions/3.10.6/envs/decp_ml_env/lib/python3.10/site-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [5]:
X_train_graph = graph_builder.create_graph(X_train_preproc)
X_train_graph

INFO:scripts.gnn_anomaly_detection:Creating graph structure from preprocessed data...
INFO:scripts.gnn_anomaly_detection:Filtered to 200716 valid contracts (removed 0 contracts with missing names)
INFO:scripts.gnn_anomaly_detection:Creating edges and edge features from preprocessed data...
INFO:scripts.gnn_anomaly_detection:Computing acheteur features from preprocessed data...
INFO:scripts.gnn_anomaly_detection:Computing titulaire features from preprocessed data...


{'nodes': [20006001000036,
  18001404502245,
  22930008201453,
  20003070800016,
  26310012500016,
  20009657600026,
  20007196700018,
  23450002300028,
  21400187700015,
  69204158500583,
  21720275300018,
  21760351300011,
  21500218900011,
  24670095900041,
  49446839000039,
  21930059700016,
  21600056200019,
  23130002100012,
  45820496300029,
  28590001500055,
  24660044900101,
  21080096700015,
  21300282700011,
  59820132500100,
  21640417800016,
  22590001801244,
  22270229200012,
  20005340300057,
  21300325400017,
  25670115200025,
  20006843500060,
  21770282800013,
  21974011500015,
  20005480700017,
  20008352500010,
  34933811100017,
  20006786600018,
  20003607700010,
  26590673500013,
  21670449400014,
  26820101900018,
  20005596000070,
  22660001300016,
  20005565500019,
  22720002900014,
  25310098600046,
  34481082500366,
  21260364100011,
  20006746000010,
  21250031800010,
  20007051400019,
  21310588500015,
  21840034900013,
  21700198100018,
  25850022200033,
 

In [9]:
import pandas as pd
import json

# Load JSON from a file
with open(os.path.join(data_path,'graph_data.json'), 'r') as f:
    X_train_graph = json.load(f)


In [6]:
graph_builder.visualize_procurement_graph(X_train_graph)

KeyboardInterrupt: 

In [11]:
# Load JSON from a file
with open(os.path.join(data_path,'graph_data.json'), 'r') as f:
    X_train_graph = json.load(f)

gnn_detector = GNNAnomalyDetector(hidden_dim=64, output_dim=32, num_layers=3)

# Scale features before creating TensorFlow graph
node_features = X_train_graph['node_features']
edge_features = X_train_graph['edge_features']

X_train_tf_graph = gnn_detector.create_tensorflow_graph(X_train_graph, node_features, edge_features)

X_train_tf_graph

INFO:scripts.gnn_anomaly_detection:Creating TensorFlow GNN graph...


AttributeError: 'list' object has no attribute 'astype'